In [57]:
#Importation des modules necessaire a notre travail
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
import numpy as np
import matplotlib.pyplot as plt 
import os 
from PIL import Image 
from torch import nn,optim
import torch.nn.functional as F 
from torch.utils.data import Dataset,DataLoader
import albumentations as A 
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from torchvision.io import read_image
import cv2
from PIL import Image
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
import torchvision

In [58]:
#chargement des données
df_train=pd.read_csv("/kaggle/input/dataset/Train.csv")
df_test=pd.read_csv("/kaggle/input/dataset/Test.csv")
print(df_train)

                   Image_id  Label
0         id_004wknd7qd.jpg  blast
1     id_004wknd7qd_rgn.jpg  blast
2         id_005sitfgr2.jpg  brown
3     id_005sitfgr2_rgn.jpg  brown
4         id_00stp9t6m6.jpg  blast
...                     ...    ...
5335  id_zz6gzk7p97_rgn.jpg  brown
5336      id_zz8ca2p67e.jpg  blast
5337  id_zz8ca2p67e_rgn.jpg  blast
5338      id_zzt8y9q0x0.jpg  brown
5339  id_zzt8y9q0x0_rgn.jpg  brown

[5340 rows x 2 columns]


In [59]:
#nombre d'occurrences de chaque classe et distribution normalisé en pourcentage
df_train['Label'].value_counts()
df_train['Label'].value_counts(normalize=True)*100

Label
blast      55.955056
brown      28.689139
healthy    15.355805
Name: proportion, dtype: float64

In [60]:
#suppression des doublons
df_train = df_train[~df_train.Image_id.str.contains('_rgn.jpg')].reset_index(drop=True)
df_train.head()

,Image_id,Label
0,id_004wknd7qd.jpg,blast
1,id_005sitfgr2.jpg,brown
2,id_00stp9t6m6.jpg,blast
3,id_012zxewnhx.jpg,blast
4,id_0186qwq2at.jpg,healthy


In [61]:
#Valeurs manquantes
df_train.isna().sum()
#df_train.info()

Image_id    0
Label       0
dtype: int64

In [62]:
#Encodage des labels
label_encoder = LabelEncoder()
df_train['Classe'] = label_encoder.fit_transform(df_train['Label'])
df_train = df_train.drop('Label', axis=1)
#blast 0 brown 1 healthy 2
print(df_train)

               Image_id  Classe
0     id_004wknd7qd.jpg       0
1     id_005sitfgr2.jpg       1
2     id_00stp9t6m6.jpg       0
3     id_012zxewnhx.jpg       0
4     id_0186qwq2at.jpg       2
...                 ...     ...
2665  id_zydzdp046u.jpg       2
2666  id_zyoowbqcm3.jpg       2
2667  id_zz6gzk7p97.jpg       1
2668  id_zz8ca2p67e.jpg       0
2669  id_zzt8y9q0x0.jpg       1

[2670 rows x 2 columns]


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


# Preparation du modele

In [63]:
#division des données en apprentissage validation
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df_train['Image_id'],df_train['Classe'], test_size=0.3, random_state=1234)

In [64]:
#transformations pour charger et prétraiter les images .
train_transform = transforms.Compose([
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=(0,180)),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])

    ])

val_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [65]:
#Dataset et dataloaders
class TV_Dataset(Dataset):

    def __init__(self, imag, labels, img_dir,transform):

        self.labels = labels
        self.img_dir = img_dir
        self.imag = imag
        self.transform = transform

    def __len__(self):

        return len(self.imag)

    def __getitem__(self,idx):
        img_path = os.path.join(self.img_dir, self.imag[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = self.transform(img)
        label = torch.tensor(self.labels[idx])
        return img, label

In [66]:
#Creation d'instance train et validation
train_dset = TV_Dataset(X_train.values, y_train.values, "/kaggle/input/image1",transform = train_transform)
valid_dset = TV_Dataset(X_val.values, y_val.values, "/kaggle/input/image1",transform = val_transform)

In [68]:
train_dataloader = DataLoader(train_dset, batch_size=150, shuffle=True)
valid_dataloader = DataLoader(valid_dset, batch_size = 150, shuffle = False)

In [69]:
#Lot d'echantillon du dataloader et dimension
train_features, train_labels = next(iter(train_dataloader))
print(f"dimension : {train_features.size()}")
print(f"étiquette: {train_labels.size()}")

dimension : torch.Size([150, 3, 224, 224])
étiquette: torch.Size([150])


In [ ]:
class Att(nn.Module):
    def __init__(self, in_features, up_factor):
        # Initialisation du module d'attention avec le nombre d'entités d'entrée (in_features)
        # et le facteur d'échelle pour l'opération d'interpolation (up_factor)
        super(Att, self).__init__()
        self.up_factor = up_factor
        self.op = nn.Conv2d(in_channels=in_features, out_channels=1, kernel_size=1, padding=0, bias=False)

    def forward(self, l, g):
        # Fonction forward, qui prend deux entrées, l (fonctionnalités locales) et g (fonctionnalités globales)
        N, C, W, H = l.size()
        
        # Interpolation de g pour ajuster sa taille à celle de l
        g = F.interpolate(g, scale_factor=self.up_factor, mode='bilinear', align_corners=False)
        
        # Combinaison des fonctionnalités locales et globales suivie d'une opération de convolution
        c = self.op(l + g)  # batch_size x 1 x W x H

        # Application de la fonction softmax pour obtenir les poids d'attention spatiaux
        a = F.softmax(c.view(N, 1, -1), dim=2).view(N, 1, W, H)
        
        # Application du masque d'attention à l pour obtenir les fonctionnalités atténuées
        g = torch.mul(a.expand_as(l), l)
        g = g.view(N, C, -1).sum(dim=2)  # batch_size x C

        # Retourne les poids d'attention spatiaux (a) et les fonctionnalités atténuées (g)
        return a, g


In [72]:
class ProjectBloc(nn.Module):
    def __init__(self, in_feat, out_feat):

        super(ProjectBloc, self).__init__()

        self.op = nn.Conv2d(in_channels=in_feat, out_channels=out_feat, kernel_size=1, padding=0, bias=False)

    def forward(self, inputs):

        return self.op(inputs)

In [73]:
import torchvision.models as models
import torch

class Att_ResNet34(nn.Module):

    def __init__(self,classes):

        super(Att_ResNet34, self).__init__()

        base_model = models.resnet34(weights = "IMAGENET1K_V1")

        layers = list(base_model.children())

        self.conv1 = nn.Sequential(*layers[0:3])

        self.conv2 = nn.Sequential(*layers[3:5])

        self.conv3 = layers[5]

        self.conv4 = layers[6]

        self.conv5 = layers[7]

        self.drp1 = nn.Dropout(0.3)
        self.drp2 = nn.Dropout(0.4)

        self.pool  = nn.AdaptiveAvgPool2d((1,1))

        self.cls = nn.Linear(in_features = 512*3, out_features = classes, bias = True)

        self.projector1 = ProjectBloc(128,512)
        self.projector2 = ProjectBloc(256,512)

        self.attn1 = Att(512,4)
        self.attn2 = Att(512,2)

    def forward(self,x):

        x = self.conv1(x)
        x = self.conv2(x)
        l1 = self.conv3(x)
        l2 = self.conv4(l1)
        x = self.conv5(l2)

        N, __, __, __ = x.size()
        g = self.pool(x).view(N,512)

        #print(x.size(),g.size(),l1.size(),l2.size())

        x1,y1 = self.attn1(self.projector1(l1),x)
        x2,y2 = self.attn2(self.projector2(l2),x)

        g = torch.cat((g,y1,y2), dim = 1)

        g = self.drp1(g)
        g = self.drp2(g)

        x = self.cls(g)

        x = F.softmax(x,dim = 1)

        return x,x1,x2


MODEL

In [74]:
x = Att_ResNet34(3)

In [75]:
import torch.optim as optim
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_labels = np.unique(y_train.values)
class_weights = compute_class_weight(class_weight = 'balanced', classes = class_labels, y = y_train.values)
class_weights = torch.from_numpy(class_weights).float()
criterion = nn.CrossEntropyLoss(class_weights)
optimizer = optim.Adam(x.parameters(), lr=0.001)

In [76]:
# Initialisation des listes pour stocker les pertes d'entraînement et de validation
train_loss = []
val_loss = []

# Initialisation de la meilleure perte avec une valeur élevée pour s'assurer qu'elle sera mise à jour
best_loss = 1000

# Boucle sur plusieurs époques (cycles d'entraînement)
for epoch in range(10):
    
    # Initialisation des pertes moyennes par époque
    Average_train_loss_per_epoch = 0.0
    Average_val_loss_per_epoch = 0.0

    # Boucle sur les lots d'entraînement
    for i, data in enumerate(train_dataloader, 0):
        
        # Initialisation de la perte par lot
        running_train_loss_per_batch = 0.0

        # Obtention des entrées et des étiquettes
        inputs, labels = data

        # Réinitialisation des gradients des paramètres
        optimizer.zero_grad()

        # Forward (passage avant), backward (rétropropagation) et optimisation
        outputs, _, _ = x(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Mise à jour des statistiques d'entraînement
        running_train_loss_per_batch += loss.item()
        Average_train_loss_per_epoch += running_train_loss_per_batch

        print(f'[{epoch + 1}, {i+1:3d}] loss: {running_train_loss_per_batch}')

    print("\n")
    print(f'epoch[{epoch + 1}] loss: {Average_train_loss_per_epoch / len(train_dataloader)}')
    train_loss.append(Average_train_loss_per_epoch / len(train_dataloader))
    print("\n")

    # Validation du modèle sans mise à jour des gradients
    with torch.no_grad():
        for i, test_data in enumerate(valid_dataloader, 0):

            # Initialisation de la perte de validation par lot
            running_val_loss_per_batch = 0.0

            # Obtention des entrées et des étiquettes de validation
            val_inputs, val_labels = test_data

            # Forward pass sur les données de validation
            y_val, _, _ = x(val_inputs)

            # Calcul de la perte de validation
            loss = criterion(y_val, val_labels)

            # Mise à jour des statistiques de perte de validation
            running_val_loss_per_batch += loss.item()
            Average_val_loss_per_epoch += running_val_loss_per_batch

    # Calcul de la perte moyenne de validation par époque
    val_loss_per_epoch = Average_val_loss_per_epoch / len(valid_dataloader)
    print(f'epoch[{epoch + 1}] val_loss: {val_loss_per_epoch}')
    val_loss.append(val_loss_per_epoch)

    # Vérification si la perte de validation actuelle est meilleure que la meilleure perte précédente
    if val_loss_per_epoch < best_loss:
        torch.save(x.state_dict(), "./poids.pth")
        best_loss = val_loss_per_epoch

print('Apprentissage Fini')


[1,   1] loss: 1.121924638748169
[1,   2] loss: 1.0798803567886353
[1,   3] loss: 1.0119836330413818
[1,   4] loss: 0.8805481195449829
[1,   5] loss: 0.9111342430114746
[1,   6] loss: 0.8992314338684082
[1,   7] loss: 0.8689140677452087
[1,   8] loss: 0.9368747472763062
[1,   9] loss: 0.900136411190033
[1,  10] loss: 0.8703016042709351
[1,  11] loss: 0.9333367943763733
[1,  12] loss: 0.8880115747451782
[1,  13] loss: 0.9497228860855103


epoch[1] loss: 0.9424615777455844


epoch[1] val_loss: 0.9360935986042023
[2,   1] loss: 0.9252828359603882
[2,   2] loss: 0.9745250940322876
[2,   3] loss: 0.8623229265213013
[2,   4] loss: 0.8296056389808655
[2,   5] loss: 0.9768058657646179
[2,   6] loss: 0.8939516544342041
[2,   7] loss: 0.9757348895072937
[2,   8] loss: 0.8848392963409424
[2,   9] loss: 0.8771483302116394
[2,  10] loss: 0.8017147183418274
[2,  11] loss: 0.8363415598869324
[2,  12] loss: 0.8940817713737488
[2,  13] loss: 0.807083010673523


epoch[2] loss: 0.8876490455407363


epoch

In [77]:
test_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [85]:
class Dataset_Test(Dataset):

    def __init__(self, images, img_rep,transform):

        self.img_rep = img_rep
        self.images = images
        self.transform = transform

    def __len__(self):

        return len(self.images)

    def __getitem__(self,idx):

        img_path = os.path.join(self.img_rep, self.images[idx])
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = self.transform(img)
        image_name = self.images[idx]

        return img,image_name


In [86]:
df_test = df_test[~df_test.Image_id.str.contains('_rgn.jpg')].reset_index(drop=True)
df_test.head()

,Image_id
0,id_00vl5wvxq3.jpg
1,id_01hu05mtch.jpg
2,id_030ln10ewn.jpg
3,id_03z57m8xht.jpg
4,id_04ngep1w4b.jpg


In [88]:
test_dataset = Dataset_Test(df_test['Image_id'].values,"/kaggle/input/image1",transform = test_transform )

In [89]:
test_dataloader = DataLoader(test_dataset, batch_size = 150, shuffle = False)

In [90]:
x = Att_ResNet34(3)
x.load_state_dict(torch.load("/kaggle/working/poids.pth"))
x.eval()

Att_ResNet34(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=T

In [92]:
import torch.nn.functional as F

df = {'Image_id': [], 'blast_prob': [], 'brown_prob': [], 'healthy_prob': []}

with torch.no_grad():
    for data in test_dataloader:
        images, names = data
        outputs, _, _ = x(images)
        
        proba = F.softmax(outputs, dim=1)

        for i in range(len(proba)):
            df['Image_id'].append(names[i])
            df['blast_prob'].append(proba[i][0].item())
            df['brown_prob'].append(proba[i][1].item())
            df['healthy_prob'].append(proba[i][2].item())

resultat = pd.DataFrame(df)


In [93]:
resultat

,Image_id,blast_prob,brown_prob,healthy_prob
0,id_00vl5wvxq3.jpg,0.576117,0.211942,0.211942
1,id_01hu05mtch.jpg,0.373918,0.393209,0.232874
2,id_030ln10ewn.jpg,0.341428,0.305732,0.352840
3,id_03z57m8xht.jpg,0.576117,0.211942,0.211942
4,id_04ngep1w4b.jpg,0.576117,0.211942,0.211942
...,...,...,...,...
1140,id_zrdlgjrq3r.jpg,0.211951,0.211976,0.576074
1141,id_zsfayxwipp.jpg,0.212985,0.215909,0.571107
1142,id_ztvp2l9k3h.jpg,0.254790,0.521993,0.223218
1143,id_zwwcma7hlt.jpg,0.576117,0.211942,0.211942


In [97]:


# Obtenez le chemin du répertoire de travail sur Kaggle
kaggle_output_dir = '/kaggle/working/'

# Spécifiez le nom du fichier CSV
output_csv_filename = 'submission_results2.csv'

# Construisez le chemin complet du fichier CSV
output_csv_path = os.path.join(kaggle_output_dir, output_csv_filename)

# Enregistrez le DataFrame dans le fichier CSV
resultat.to_csv(output_csv_path, index=False)
